# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [5]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [6]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [7]:

#problems reading file 
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load((open(filename, 'rb')))
    target = [i] * len(data)
    return data, target


In [8]:
# load all the data
all_data = []
all_target = []
for i in range(len(classes)):
    data, target = load_pose_data(i)
    if data:
        all_data += data
        all_target += target
print('Total number of data:', len(all_data))
#print(all_data)

ValueError: could not convert string to float

In [9]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
print(len(training_data))

0


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [10]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [11]:
# YOUR CODE HERE
train = []
target = []
for i in range(len(training_data)):
    train.append(all_data[training_data[i]])
    target.append(all_target[training_data[i]])
clf.fit(train, target) 

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

### predicting

In [12]:
clf.predict(all_data[-1:]), all_target[-1:]

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [13]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [14]:
expected = target
predicted = clf.predict(train)
# YOUR CODE HERE
evaluate(expected, predicted)

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## 4. Evaluate on the test data

In [15]:
expected = []
predicted = []
# YOUR CODE HERE
#expected = [all_target[i] for i in training_data]
#ndata = [all_data[i] for i in training_data]
for i in training_data:
    expected = all_target[i]
    test = all_data[i]

predicted = clf.predict(test)
evaluate(expected, predicted)

NameError: name 'test' is not defined

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [16]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [ ]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 43: character maps to <undefined>